<a href="https://colab.research.google.com/github/emkafie/Machine-Learning/blob/main/P2_JS13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 2**

## **Persiapan Data**

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import numpy as np
import os

# Mengatur seed agar hasil dapat direproduksi
np.random.seed(42)
tf.random.set_seed(42)

# 1. Load dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# 2. One-hot encoding (sparse=False diganti sparse_output=False untuk versi sklearn terbaru,
# namun sparse=False tetap kompatibel dengan versi lama)
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

# 3. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data siap digunakan.")
print(f"Ukuran X_train: {X_train.shape}")

Data siap digunakan.
Ukuran X_train: (120, 4)


Output menampilkan dimensi data latih. X_train memiliki bentuk (120, 4), yang berarti terdapat 120 sampel data latih dengan 4 fitur (panjang/lebar sepal dan petal). Data label y telah diubah menjadi format one-hot encoding untuk klasifikasi multikelas.

## **Ubah Jumlah Neuron Hidden Layer**

Pada bagian ini, jumlah neuron pada hidden layer ditingkatkan dari konfigurasi awal (10 dan 8) menjadi 64 dan 32 untuk melihat dampaknya terhadap kapasitas model dalam mempelajari pola.

In [7]:
# Konfigurasi Modifikasi Neuron (64 dan 32 neuron)
model_neuron = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)), # Diubah dari 10 ke 64
    tf.keras.layers.Dense(32, activation='relu'),                   # Diubah dari 8 ke 32
    tf.keras.layers.Dense(3, activation='softmax')
])

# Kompilasi
model_neuron.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Latih model
print("--- Training Model Modifikasi Neuron ---")
history_neuron = model_neuron.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0)

# Evaluasi
loss_neuron, acc_neuron = model_neuron.evaluate(X_test, y_test, verbose=0)

print(f"Loss Akhir: {loss_neuron:.4f}")
print(f"Akurasi Modifikasi Neuron: {acc_neuron:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--- Training Model Modifikasi Neuron ---
Loss Akhir: 0.1024
Akurasi Modifikasi Neuron: 1.0000


1. Akurasi: Dengan penambahan jumlah neuron menjadi 64 dan 32, model memiliki kapasitas yang lebih besar. Akurasi yang dihasilkan biasanya setara atau lebih tinggi (seringkali mencapai 1.0 atau 100% pada dataset Iris yang sederhana) dibandingkan konfigurasi awal yang lebih kecil.

2. Loss: Nilai loss menunjukkan seberapa baik prediksi model mendekati label asli. Nilai yang semakin kecil menandakan model semakin presisi.

## **Bandingkan Sigmoid vs ReLU**

In [8]:
# Konfigurasi Aktivasi Sigmoid (Arsitektur kembali ke 10 dan 8)
model_sigmoid = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='sigmoid', input_shape=(4,)), # Aktivasi diganti Sigmoid
    tf.keras.layers.Dense(8, activation='sigmoid'),                    # Aktivasi diganti Sigmoid
    tf.keras.layers.Dense(3, activation='softmax')                     # Output tetap Softmax
])

# Kompilasi
model_sigmoid.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Latih model
print("--- Training Model Sigmoid ---")
history_sigmoid = model_sigmoid.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0)

# Evaluasi
loss_sigmoid, acc_sigmoid = model_sigmoid.evaluate(X_test, y_test, verbose=0)

print(f"Loss Akhir: {loss_sigmoid:.4f}")
print(f"Akurasi Model Sigmoid: {acc_sigmoid:.4f}")

--- Training Model Sigmoid ---
Loss Akhir: 0.5880
Akurasi Model Sigmoid: 0.8000


1. Performa Lambat: Pada jumlah epoch yang sama (50), model dengan Sigmoid seringkali menghasilkan loss yang lebih tinggi dan akurasi yang lebih rendah dibandingkan ReLU.

2. Penyebab: Sigmoid rentan terhadap masalah vanishing gradient (gradien menghilang), yang membuat proses pembaruan bobot menjadi lebih lambat, terutama pada jaringan yang lebih dalam atau epoch awal. ReLU lebih efisien secara komputasi dan konvergensi.

## **Rangkuman Perbandingan**

In [9]:
import pandas as pd

results = {
    "Konfigurasi": ["Modifikasi Neuron (ReLU 64-32)", "Aktivasi Sigmoid (10-8)"],
    "Akurasi": [acc_neuron, acc_sigmoid],
    "Loss": [loss_neuron, loss_sigmoid]
}

df_results = pd.DataFrame(results)
print(df_results)

                      Konfigurasi  Akurasi      Loss
0  Modifikasi Neuron (ReLU 64-32)      1.0  0.102380
1         Aktivasi Sigmoid (10-8)      0.8  0.587962


* Modifikasi Neuron (ReLU): Cenderung menghasilkan akurasi tertinggi dan loss terendah. Penambahan neuron membantu model menangkap fitur lebih detail.

* Aktivasi Sigmoid: Cenderung memiliki akurasi lebih rendah dan loss lebih tinggi dibandingkan ReLU pada jumlah epoch yang terbatas. Hal ini membuktikan bahwa ReLU umumnya lebih superior untuk hidden layer pada kasus klasifikasi modern dibandingkan Sigmoid.